In [1]:
import pandas as pd

## Antecedentes - Importando Bases
#### 1. Planilla:
* Importancia: 
    * Base de colaboradores y el area de donde pertenecen desde el 2018 al 2021 
    * Es el archivo madre desde donde trabajar porque tiene todos los colaboradores
* Variables a usar:
    * MES, COD, Cargo, Oficina, Producto, Territorio y Zona
#### 2. Entrevistas Ceses
* Importancia: 
    * Reporte de Encuestas hechas a cesados en el proceso de salida de la empresa
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 3. Programa de mejora
* Importancia: 
    * Base de colaboradores que entraron al programa de mejora y su tiempo de permanencia. si no cumple con el programa el colaborador es invitado a retirarse.
* Variables a usar:
    * Codigo, motivo, fecha inicio programa, status programa
#### 4. Memorandum
* Importancia: 
    * Lista de memorandums aplicados a los colaboradores por diferentes tipos de falta
* Variables a usar:
    * Codigo, Cargo, Fecha de memo, Reporte
#### 5. Vacaciones
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * Codigo, cargo, Dias pendientes
#### 6. Desacanso Medico
* Importancia: 
    * Nro de dias otrogados a los colaboradores por descanso medico por mes
* Variables a usar:
    * Codigo, cargo, Dias de descanso medico
#### 7. Bonos Venta Individual y grupal
* Importancia: 
    * Bono otorgado por cumplir ciertas reglas y metas en las ventas del mes en individual y Grupal
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 8. Desempeno
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 9. Reconocimientos
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 10. Hijos total
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 11. Info personal
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 12. Recategorizaciones
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 13. Cobertura Personal - Oficina
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 14. Antiguedad de Oficina
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo
#### 15. GPTW por oficina
* Importancia: 
    * Nro de vacaciones acumuladas por no tomarlas cuando correspondia, al ano se se deberian tomar 30 dias de vacaciones
* Variables a usar:
    * CESE, Ingreso, COD, Cargo


In [34]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=csv&gid='
    url = (url).replace(edit, export)
    df = pd.read_csv(url)
    return df

In [35]:
planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
entrevistas = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
bonos_venta_g = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()
datos_personales = extrae_drive()
recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()